In [1]:
import pandas as pd
import numpy as np


In [2]:
from lib.process import processor
from lib.model import extractor,recommendor

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Part 1: Data engineering

## Part 1.1 Read data

In [3]:
df = pd.read_csv('data/all_stocks_5yr.csv',index_col=False)
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


## Part 1.2 Generate raw features ("return","vol_change")

In [4]:
myProcessor = processor()

In [5]:
dfFeatured = myProcessor.generate_features(df)
dfFeatured.head()

,index,date,open,high,low,close,volume,Name,return,vol_change
1,1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,0.020055,-0.053423
2,2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,0.013315,0.093035
3,3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,-0.026603,-0.207954
4,4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,0.047891,-0.678183
5,5,2013-02-15,13.93,14.61,13.93,14.50,15628000,AAL,-0.035172,1.039922


## Part 1.3 Generate monthly features

In [6]:
dfFeaturedMontly = myProcessor.generate_monthly_features(dfFeatured)

In [7]:
dfFeaturedMontly.head()

level_0  index        date   open     high    low  close   volume  \
Name                                                                        
A    1  2013-03  71625  2013-03-01  41.18  41.9800  40.73  41.93  3089323   
     2  2013-04  71645  2013-04-01  41.83  41.9771  40.79  40.93  2541331   
     3  2013-05  71667  2013-05-01  41.42  41.7400  41.26  41.31  2726213   
     4  2013-06  71689  2013-06-03  45.65  45.8400  45.04  45.51  3677473   
     5  2013-07  71709  2013-07-01  43.05  43.7700  42.91  43.59  4283821   

       Name    return  vol_change  
Name                               
A    1    A  0.075125   -0.409335  
     2    A  0.024432    0.215632  
     3    A -0.009199   -0.067816  
     4    A -0.092287   -0.258672  
     5    A  0.044047   -0.141544

## Part 1.3 Generate CNN features (latent features)

In [8]:
x,y,ID_2_stock = myProcessor.generate_cnn_input(dfFeaturedMontly)

D:\DBC\Data Project预习课件0127更新\项⽬4：推荐系统应用financial_product_recommendation\financial_product_recommendation\lib\process.py:39: FutureWarning: 'Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  df_X_Y = df.groupby('Name').apply(lambda x: self.cnn_process(x,rollSize=12))


In [9]:
recent_xs = dfFeaturedMontly.groupby("Name").apply(lambda x: x[['return','vol_change']].values[-11:])
recent_x = []
for stock in ID_2_stock.values():
    recent_x.append(recent_xs[stock])
recent_x = np.array(recent_x)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


# part 2: Train models

## Part 2.1: Train extractor

In [10]:
myExtractor = extractor()
myExtractor.train(x,y,n_timesteps=11,n_features=2,verbose=1,batch_size=2000,epochs=300,n_outputs=1)

you will need to input <brew install graphviz> in the terminal to plot the graph
Train on 16433 samples, validate on 7043 samples
Epoch 1/300
16433/16433 [==============================] - 1s 47us/step - loss: 0.0088 - val_loss: 0.0047
Epoch 2/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0054 - val_loss: 0.0046
Epoch 3/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0054 - val_loss: 0.0046
Epoch 4/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 5/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 6/300
16433/16433 [==============================] - 0s 20us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 7/300
16433/16433 [==============================] - 0s 20us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 8/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 9/300
16433/1643

16433/16433 [==============================] - 0s 22us/step - loss: 0.0043 - val_loss: 0.0046
Epoch 77/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0043 - val_loss: 0.0045
Epoch 78/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0043 - val_loss: 0.0047
Epoch 79/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0043 - val_loss: 0.0045
Epoch 80/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0043 - val_loss: 0.0046
Epoch 81/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0042 - val_loss: 0.0046
Epoch 82/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0042 - val_loss: 0.0047
Epoch 83/300
16433/16433 [==============================] - 0s 23us/step - loss: 0.0042 - val_loss: 0.0046
Epoch 84/300
16433/16433 [==============================] - 0s 25us/step - loss: 0.0042 - val_loss: 0.0046
Epoch 85/300
16433/16433 [========================

16433/16433 [==============================] - 0s 24us/step - loss: 0.0033 - val_loss: 0.0052
Epoch 153/300
16433/16433 [==============================] - 0s 25us/step - loss: 0.0033 - val_loss: 0.0052
Epoch 154/300
16433/16433 [==============================] - 0s 24us/step - loss: 0.0033 - val_loss: 0.0051
Epoch 155/300
16433/16433 [==============================] - 0s 23us/step - loss: 0.0033 - val_loss: 0.0053
Epoch 156/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0033 - val_loss: 0.0053
Epoch 157/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0033 - val_loss: 0.0052
Epoch 158/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0032 - val_loss: 0.0052
Epoch 159/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0033 - val_loss: 0.0051
Epoch 160/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0032 - val_loss: 0.0052
Epoch 161/300
16433/16433 [===============

Epoch 228/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0027 - val_loss: 0.0057
Epoch 229/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0027 - val_loss: 0.0057
Epoch 230/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0027 - val_loss: 0.0057
Epoch 231/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0027 - val_loss: 0.0057
Epoch 232/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0027 - val_loss: 0.0058
Epoch 233/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0027 - val_loss: 0.0058
Epoch 234/300
16433/16433 [==============================] - 0s 22us/step - loss: 0.0027 - val_loss: 0.0058
Epoch 235/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0026 - val_loss: 0.0058
Epoch 236/300
16433/16433 [==============================] - 0s 21us/step - loss: 0.0027 - val_loss: 0.0059
Epoch 237/300
16433/16433 [=

In [11]:
featureMatrix = myExtractor.get_feature_matrix(recent_x)
featureMatrix.shape

(500, 10)

## Part 2.2: Train recommendor

In [12]:
myRecommendor = recommendor()

In [13]:
myRecommendor.train(featureMatrix=featureMatrix)

Instructions for updating:
dim is deprecated, use axis instead


In [14]:
myRecommendor.similarityMatrix[:5,:5]

array([[0.9999999 , 0.99999994, 0.63825625, 0.99999994, 0.9999999 ],
       [0.99999994, 1.        , 0.6382563 , 1.        , 0.99999994],
       [0.63825625, 0.6382563 , 0.9999999 , 0.6382563 , 0.63825625],
       [0.99999994, 1.        , 0.6382563 , 1.        , 0.99999994],
       [0.9999999 , 0.99999994, 0.63825625, 0.99999994, 0.9999999 ]],
      dtype=float32)

# Part 3 Recommend

In [15]:
myRecommendor.recommend_to_user(productID=200,top=3)

For productID: 200 ,similar 3 product are: [ 85 119 130], similarity values [1.0000002 1.0000002 1.0000002]


(array([1.0000002, 1.0000002, 1.0000002], dtype=float32),
 array([ 85, 119, 130]))